## Knapsack Problem 背包问题


Ortools专门为解决背包问题提供了一个接口（knapsack_solver）。因为背包问题应用十分广泛，且常常作为一些优化问题的子问题出现（比如列生成算法），因此备受重视。

> 背包问题的描述是：需要将一组具有给定值和尺寸（例如重量或体积）的物品装入具有一定容量的容器中。如果物品的总尺寸超过容量，则无法将它们全部打包。在这种情况下，找出能放进容器并使得价值最大的物品集合。

$$\max \sum^{M}_1 v_i x_i \\

s.t. \begin{aligned}\begin{equation*}
\begin{cases}
\sum^{M}_{1} w_ix_i \leq K \\
x_i \in \{0, 1\}
\end{cases}
\end{equation*}\end{aligned}$$

其中 $M$ 表示所有物品的集合，$v_i$表示第$i$个物品的价值，$w_i$表示第i个物品的价值，$x_i$是决策变量：一个 0-1 变量，表示“是否把第i个物品放到包里”。


<img src="./assets/figures/knapsack.png" alt="image" width="300" height="300" align="center" />





In [1]:
from ortools.algorithms.python import knapsack_solver
def main():
    # 创建求解器
    solver = knapsack_solver.KnapsackSolver(
        knapsack_solver.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
        # 属性 KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER 
        # 告诉求解器使用分支定界限（Branch and Bound）算法来解决问题。
        "KnapsackExample",
    )
    values = [
        # fmt:off
      360, 83, 59, 130, 431, 67, 230, 52, 93, 125, 670, 892, 600, 38, 48, 147,
      78, 256, 63, 17, 120, 164, 432, 35, 92, 110, 22, 42, 50, 323, 514, 28,
      87, 73, 78, 15, 26, 78, 210, 36, 85, 189, 274, 43, 33, 10, 19, 389, 276,
      312
        # fmt:on
    ]
    weights = [
        # fmt: off
      [7, 0, 30, 22, 80, 94, 11, 81, 70, 64, 59, 18, 0, 36, 3, 8, 15, 42, 9, 0,
       42, 47, 52, 32, 26, 48, 55, 6, 29, 84, 2, 4, 18, 56, 7, 29, 93, 44, 71,
       3, 86, 66, 31, 65, 0, 79, 20, 65, 52, 13],
        # fmt: on
    ]
    capacities = [850]

    solver.init(values, weights, capacities)
    computed_value = solver.solve()

    packed_items = []
    packed_weights = []
    total_weight = 0
    
    # 最优解的总值为 computed_value ，与本例中的总重量相同。
    # 然后，程序获取解决方案中包装物料的索引，如下所示：
    
    print("Total value =", computed_value)
    for i in range(len(values)):
        if solver.best_solution_contains(i):
            packed_items.append(i)
            packed_weights.append(weights[0][i])
            total_weight += weights[0][i]
    print("Total weight:", total_weight)
    print("Packed items:", packed_items)
    print("Packed_weights:", packed_weights)


if __name__ == "__main__":
    main()

Total value = 7534
Total weight: 850
Packed items: [0, 1, 3, 4, 6, 10, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 24, 27, 28, 29, 30, 31, 32, 34, 38, 39, 41, 42, 44, 47, 48, 49]
Packed_weights: [7, 0, 22, 80, 11, 59, 18, 0, 3, 8, 15, 42, 9, 0, 47, 52, 26, 6, 29, 84, 2, 4, 18, 7, 71, 3, 66, 31, 0, 65, 52, 13]


## Multi Knapsack Problem

和最基本的背包问题一样，我们从具有不同重量和价值的物品集合开始。

$$\max \sum^B\sum^{M} v_i x_{ib} \\

s.t. \begin{aligned}\begin{equation*}
\begin{cases}
\sum^{M} w_ix_{ib} \leq K_b, \hspace{5pt} \forall b \in B \\
x_{ib} \in \{0, 1\}
\end{cases}
\end{equation*}\end{aligned}$$

问题是将物品的一个子集合打包到多个箱子中，每个箱的最大容量均给定，使得所有的箱子装的物品的总价值最大。当规模过大的时候，求解难度会快速上升，不过ortools提供了快速的求解方法。

In [ ]:
"""Solves a multiple knapsack problem using the CP-SAT solver."""
from ortools.sat.python import cp_model


def main():
    data = {}
    data["weights"] = [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36, 43, 51, 39, 52]
    data["values"] = [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25, 25, 42, 39, 32]
    assert len(data["weights"]) == len(data["values"])
    data["num_items"] = len(data["weights"])
    data["all_items"] = range(data["num_items"])

    data["bin_capacities"] = [97, 101, 123, 112, 121]
    data["num_bins"] = len(data["bin_capacities"])
    data["all_bins"] = range(data["num_bins"])

    model = cp_model.CpModel()

    # Variables.
    # 决策变量：x[i, b] 是否把i物品放入b箱子中。
    
    x = {}
    for i in data["all_items"]:
        for b in data["all_bins"]:
            x[i, b] = model.NewBoolVar(f"x_{i}_{b}")

    # 约束条件
    # 每个物品只能在箱子里出现一次
    for i in data["all_items"]:
        model.AddAtMostOne(x[i, b] for b in data["all_bins"])
    # AddAtMostOne(list(BoolVar)): 列表里的0-1变量至多出现一次

    # 每个箱子里放的物品的重量不能超过箱子的重量
    
    for b in data["all_bins"]:
        model.Add(
            sum(x[i, b] * data["weights"][i] for i in data["all_items"])
            <= data["bin_capacities"][b]
        )

    # 目标函数
    # 最大化装进去的东西的价值
    
    objective = []
    for i in data["all_items"]:
        for b in data["all_bins"]:
            objective.append(cp_model.LinearExpr.Term(x[i, b], data["values"][i]))
    model.Maximize(cp_model.LinearExpr.Sum(objective))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        print(f"Total packed value: {solver.ObjectiveValue()}")
        total_weight = 0
        for b in data["all_bins"]:
            print(f"Bin {b}")
            bin_weight = 0
            bin_value = 0
            for i in data["all_items"]:
                if solver.Value(x[i, b]) > 0:
                    print(
                        f"Item {i} weight: {data['weights'][i]} value: {data['values'][i]}"
                    )
                    bin_weight += data["weights"][i]
                    bin_value += data["values"][i]
            print(f"Packed bin weight: {bin_weight}")
            print(f"Packed bin value: {bin_value}\n")
            total_weight += bin_weight
        print(f"Total packed weight: {total_weight}")
    else:
        print("The problem does not have an optimal solution.")


if __name__ == "__main__":
    main()

Total packed value: 488.0
Bin 0
Item 4 weight: 36 value: 35
Item 9 weight: 24 value: 35
Item 14 weight: 36 value: 25
Packed bin weight: 96
Packed bin value: 95

Bin 1
Item 5 weight: 48 value: 30
Item 7 weight: 42 value: 40
Packed bin weight: 90
Packed bin value: 70

Bin 2
Item 3 weight: 36 value: 50
Item 8 weight: 36 value: 30
Item 16 weight: 51 value: 42
Packed bin weight: 123
Packed bin value: 122

Bin 3
Item 2 weight: 42 value: 25
Item 10 weight: 30 value: 45
Item 17 weight: 39 value: 39
Packed bin weight: 111
Packed bin value: 109

Bin 4
Item 1 weight: 30 value: 30
Item 13 weight: 36 value: 30
Item 18 weight: 52 value: 32
Packed bin weight: 118
Packed bin value: 92

Total packed weight: 538


## Bin Packing Problem

与多重背包问题（Multiple Knapsack Problem）类似，装箱问题（Bin Packing Problem）也涉及将物品装入箱子。然而，装箱问题的目标不同：它旨在寻找能够装下所有物品的**最少箱子数量**。

以下内容总结了这两个问题之间的区别：

- **多重背包问题**：将物品的子集装入固定数量且容量各不相同的箱子中，以使装入物品的总价值最大化。
- **装箱问题**：假设拥有足够多的、容量相同的箱子，找出能容纳所有物品的最少箱子数量。在这个问题中，物品不被赋予价值，因为其优化目标并不涉及物品价值。

下面的示例将展示如何求解一个装箱问题。

In [7]:
from ortools.linear_solver import pywraplp
import random

def create_data_model():
    """Create the data for the example."""
    data = {}
    # weights = [48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30]
    random.seed(2025)
    # 生成100个在20到50之间的随机整数
    weights = [random.randint(20, 50) for _ in range(100)]


    data["weights"] = weights
    data["items"] = list(range(len(weights)))
    data["bins"] = data["items"]
    data["bin_capacity"] = 200
    return data



def main():
    data = create_data_model()

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver("SCIP")

    if not solver:
        return

    # Variables
    # x[i, j] = 1 if item i is packed in bin j.
    x = {}
    for i in data["items"]:
        for j in data["bins"]:
            x[(i, j)] = solver.IntVar(0, 1, "x_%i_%i" % (i, j))

    # y[j] = 1 if bin j is used.
    y = {}
    for j in data["bins"]:
        y[j] = solver.IntVar(0, 1, "y[%i]" % j)

    # Constraints
    # Each item must be in exactly one bin.
    for i in data["items"]:
        solver.Add(sum(x[i, j] for j in data["bins"]) == 1)

    # The amount packed in each bin cannot exceed its capacity.
    for j in data["bins"]:
        solver.Add(
            sum(x[(i, j)] * data["weights"][i] for i in data["items"])
            <= y[j] * data["bin_capacity"]
        )

    # Objective: minimize the number of bins used.
    solver.Minimize(solver.Sum([y[j] for j in data["bins"]]))

    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0
        for j in data["bins"]:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_weight = 0
                for i in data["items"]:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                        bin_weight += data["weights"][i]
                if bin_items:
                    num_bins += 1
                    print("Bin number", j)
                    print("  Items packed:", bin_items)
                    print("  Total weight:", bin_weight)
                    print()
        print()
        print("Number of bins used:", num_bins)
        print("Time = ", solver.WallTime(), " milliseconds")
    else:
        print("The problem does not have an optimal solution.")


if __name__ == "__main__":
    main()
 

Solving with SCIP 9.0.0 [LP solver: Glop 9.10]
Bin number 0
  Items packed: [0, 1, 2, 3, 4, 8]
  Total weight: 200

Bin number 1
  Items packed: [5, 6, 7, 9, 10, 13]
  Total weight: 196

Bin number 2
  Items packed: [11, 12, 14, 15, 16, 17, 18]
  Total weight: 196

Bin number 3
  Items packed: [19, 20, 21, 22, 23, 24, 26]
  Total weight: 199

Bin number 4
  Items packed: [25, 27, 28, 29, 30, 35]
  Total weight: 198

Bin number 5
  Items packed: [31, 32, 33, 34, 36]
  Total weight: 186

Bin number 6
  Items packed: [37, 38, 39, 40, 42]
  Total weight: 199

Bin number 7
  Items packed: [41, 43, 44, 45, 46]
  Total weight: 190

Bin number 8
  Items packed: [47, 48, 49, 50, 51, 52]
  Total weight: 194

Bin number 9
  Items packed: [53, 54, 55, 56, 57, 58, 60]
  Total weight: 183

Bin number 10
  Items packed: [59, 61, 62, 63, 64, 72]
  Total weight: 198

Bin number 11
  Items packed: [65, 66, 67, 68, 69]
  Total weight: 186

Bin number 12
  Items packed: [70, 71, 73, 74, 84]
  Total weight